In [ ]:
# @title Install Transformers and Retrieve Probing Data
from IPython.display import clear_output
!pip install transformers
!git clone https://github.com/facebookresearch/SentEval
from google.colab import drive
drive.mount('/content/drive')
clear_output()

In [ ]:
# @title Import Requirements

from transformers import (
    BertConfig,
    BertTokenizer,
    TFBertModel,
    BertModel,
    glue_processors,
    glue_convert_examples_to_features,
    set_seed
)
from transformers.optimization_tf import create_optimizer

import tensorflow as tf
import tensorflow_datasets
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
import matplotlib.pyplot as plt
import copy 
import os
import pandas as pd
import torch.nn as nn 

In [ ]:
# @title Hyperparameters
BATCH_SIZE =  32# @param {type:"integer"}
EPOCHS =  10#@param {type:"integer"}
MAX_LENGTH =   64#@param {type:"integer"}

TASK = "qqp" #@param ["cola", "sst2", "mrpc", "sts", "qqp", "mnli", "qnli", "rte"]

MODEL_SIZE = "full"#@param ["full", "7k", "2.5k", "1k"]
PROBE = "BigramShift" #@param ['Length','BigramShift', 'TopConst', 'Tense','SubjNumber', 'ObjNumber', 'OddManOut', 'CoordinationInversion']
MODEL_SEED = "42" #@param [42, 123, 1234]
PROBE_SEED = "60" #@param [40, 50, 60]
num_labels = 2

if PROBE == "TopConst":
  num_labels = 20
elif PROBE == 'Length':
    num_labels = 6

LEARNING_RATE =  3e-4 #@param {type:"number"}
WARMUP_RATIO =   0.1 #@param {type:"number"}
LAYER = "12" #@param [1,2,3,4, 5,6, 7,8, 9,10, 11, 12] 
LAYER = int(LAYER)

# Loading fine-tuned models. Replace this line with the path to your saved models directory
SAVED_MODELS_DIR = "/content/drive/MyDrive/Fine-Tuned-Models/" + TASK + '-' + MODEL_SIZE + '-' + str(MODEL_SEED)

# List of tasks
DATA_NAME = ""
if PROBE == "Length":
  DATA_NAME = "sentence_length.txt"
elif PROBE == "BigramShift":
  DATA_NAME = "bigram_shift.txt"
elif PROBE == "TopConst":
  DATA_NAME = "top_constituents.txt"
elif PROBE == "Tense":
  DATA_NAME = "past_present.txt"
elif PROBE == "SubjNumber":
  DATA_NAME = "subj_number.txt"
elif PROBE == "ObjNumber":
  DATA_NAME = "obj_number.txt"
elif PROBE == "OddManOut":
  DATA_NAME = "odd_man_out.txt"
elif PROBE == "CoordinationInversion":
  DATA_NAME = "coordination_inversion.txt"

set_seed(int(PROBE_SEED))

In [ ]:
# @title Loading Data Set
df = pd.read_csv("/content/SentEval/data/probing/" + DATA_NAME, sep = '\t', header=None)

if PROBE != 'Length':
  df[1] = df[1].factorize()[0]

df_train = df[df[0]=="tr"]
df_val = df[df[0]=="va"]
df_test = df[df[0]=="te"]

In [ ]:
# @title Probe model

class ProbeModel(tf.keras.Model):
  def __init__(self, bert_model, num_labels, layer, *inputs, **kwargs):
    super(ProbeModel, self).__init__(name="ProbeModel")
    self.bert = bert_model
    self.bert.trainable = False
    self.dropout = tf.keras.layers.Dropout(0.1)
    self.layer = layer
    self.classifier = tf.keras.layers.Dense(
                      num_labels,
                      kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02),
                      name="classifier")
    
  
  def call(self, inputs, **kwargs):

    outputs = self.bert(inputs, **kwargs)
    pooled_out = outputs[2][self.layer]
    pooled_out = pooled_out[:,0,:]
    # pooled_out = outputs[1]
    
    droped_out = self.dropout(pooled_out, training=kwargs.get("training", False))
    output = self.classifier(droped_out)
    return output

In [ ]:
# @title Tokenizer
def tokenize(df, tokenizer, MAX_LENGTH):
    input_ids, input_masks, input_segments = [],[],[]
    for i in range(len(df)):
        inputs = tokenizer.encode_plus(df[2].iloc[i], add_special_tokens=True, 
                                       return_attention_mask=True, return_token_type_ids=True, max_length = MAX_LENGTH,
                                       pad_to_max_length=True, truncation =True )

        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])  

    return [tf.cast(input_ids,tf.int32),tf.cast(input_masks,tf.int32), tf.cast(input_segments,tf.int32)]

In [ ]:
# @title Load Model

casing = "bert-base-uncased" 
config = BertConfig.from_pretrained(casing, num_labels=num_labels)
config.output_hidden_states = True
tokenizer = BertTokenizer.from_pretrained(casing)

bert_model = TFBertModel.from_pretrained(SAVED_MODELS_DIR, from_pt=True, config = config)
bert_model.output_hidden_states=True
model = ProbeModel(bert_model, num_labels, LAYER)


In [ ]:
# @title Make dataset ready 

train_dataset = tokenize(df_train, tokenizer, MAX_LENGTH)
valid_dataset = tokenize(df_val, tokenizer, MAX_LENGTH)
test_dataset  = tokenize(df_test, tokenizer, MAX_LENGTH)
  

train_steps = int(np.ceil(len(df_train) / BATCH_SIZE))
valid_steps = int(np.ceil((len(df_val)) / BATCH_SIZE))
test_steps = int(np.ceil(len(df_test) / BATCH_SIZE))

clear_output()
print("completed")

In [ ]:
# @title Freeze BERT
model.bert.bert.embeddings.trainable = False
model.bert.bert.pooler.trainable = False
for i in range(config.num_hidden_layers):
  model.bert.bert.encoder.layer[i].attention.self_attention.trainable = False
  model.bert.bert.encoder.layer[i].attention.dense_output.dense.trainable = False
  model.bert.bert.encoder.layer[i].attention.dense_output.LayerNorm.trainable = False
  model.bert.bert.encoder.layer[i].intermediate.trainable = False
  model.bert.bert.encoder.layer[i].bert_output.dense.trainable = False
  model.bert.bert.encoder.layer[i].bert_output.LayerNorm.trainable = False

In [ ]:
# @title Metrics, Loss & Optimizer

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt, scheduler = create_optimizer(init_lr=LEARNING_RATE,
                       num_train_steps=train_steps * EPOCHS,
                       num_warmup_steps=int(train_steps * EPOCHS * WARMUP_RATIO),
                       adam_epsilon=1e-6,
                       weight_decay_rate = 0)
optimizer = tf.keras.optimizers.Adam(3e-4)
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])


In [ ]:
# @title Callback
class ModelCheckpoint(tf.keras.callbacks.Callback):
  def __init__(self, monitor, save_path):
    super(ModelCheckpoint, self).__init__()
    self.monitor = monitor
    self.save_path = save_path
    self.bestScore = -np.Inf
    self.bestLoss = np.Inf

  def on_epoch_end(self, epoch, logs):
    score = logs.get(self.monitor)
    loss = logs.get("val_loss")
    if score > self.bestScore or (score == self.bestScore and loss < self.bestLoss):
      path = os.path.join(TASK, str(epoch+1))
      os.makedirs(path)
      self.model.save_weights(path+'/best_weights.h5')
      self.bestScore = score
      self.bestLoss = loss
      print("\nModel saved as the best model")

monitor = "val_accuracy"
checkpoint = ModelCheckpoint(monitor, SAVED_MODELS_DIR)


In [ ]:
# @title Training
history = model.fit(
    train_dataset,
    df_train[1],
    epochs=EPOCHS,
    batch_size = BATCH_SIZE,
    validation_data=(valid_dataset, df_val[1]),
    callbacks=[checkpoint]

)
print(model.summary())

In [ ]:
# @title Retrieving Best Model
import os 
list_of_dirs = os.listdir('/content/' + TASK)
 
# using map() to
# perform conversion
final_list = list(map(int, list_of_dirs))
best_model = max(final_list)

In [ ]:
# @title Loading and Evaluating the Best Model

model_path ="/content/"+ TASK + '/' + str(best_model) + '/best_weights.h5'

model.load_weights(model_path)

model.evaluate(test_dataset, df_test[1])

In [ ]:
#@title Removing the Checkpoints

import shutil

shutil.rmtree('/content/' + TASK + '/')

print(PROBE + ':')
print(TASK + ' layer: ' + str(LAYER) + ' with size:' + MODEL_SIZE + '  is done')